In [53]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;


In [54]:
import json

with open('parsed_data/news_flat.json') as f:
    data = json.load(f)

data_new = [[w.lower() for w in line] for line in data]
    
print(data_new)

[['next', 'democratic', 'debate', 'start', 'tier', 'candidate', 'calendar', 'front', 'runner', 'life', 'campaign', 'rationale', 'race', 'joseph', 'r.', 'biden', 'jr.'], ['ivy', 'mix', 'cocktail', 'expert', 'sundays', 'name', 'day', 'award', 'bartender', 'politic', 'drink'], ['boris', 'johnson', 'party', 'loyalists', 'trump', 'republican', 'president', 'trump', 'agenda', 'britain', 'rebel', 'conservative', 'party', 'insurrection'], ['ex', 'uk', 'minister', 'johnson', 'brexit', 'deal', 'minister', 'prime', 'minister', 'boris', 'johnson', 'cabinet', 'government', 'effort', 'brexit', 'agreement', 'european', 'union', 'johnson', 'insistence', 'deal'], ['sc', 'congressman', 'mark', 'sanford', 'gop', 'bid', 'trump', 'former', 'south', 'carolina', 'governor', 'mark', 'sanford', 'president', 'donald', 'trump', 'republican', 'party', 'nomination', 'race'], ['gop', 'survivor', 'gabby', 'giffords', 'husband', 'track', 'official', 'mark', 'kelly', 'arizona', 'candidate', 'u.s.', 'senate', 'husband'

In [55]:
id2word = gensim.corpora.Dictionary(data_new)

In [61]:
corpus = [id2word.doc2bow(text) for text in data_new];
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10);

In [62]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)

In [63]:
import pandas as pd
get_lda_topics(lda, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,warren,brexit,trump,trump,trump,trump,trump,u.s.,china,trump
1,biden,deal,president,president,president,impeachment,news,news,leader,warren
2,candidate,union,donald,impeachment,impeachment,president,president,election,kong,politic
3,debate,leader,impeachment,donald,ukraine,house,impeachment,president,hong,week
4,court,european,house,house,biden,democrats,ukraine,china,diplomat,elizabeth
5,politic,campaign,china,ukraine,news,senate,new,sanders,protest,facebook
6,campaign,president,inquiry,biden,house,’s,poll,candidate,nba,democrats
7,trump,carter,white,debate,romney,u.s.,u.s.,trump,politic,harris
8,elizabeth,time,giuliani,democrats,campaign,politic,video,bernie,trump,kamala
9,’s,uk,’s,election,joe,inquiry,york,impeachment,power,tax


In [59]:
num_topics = 10

data_new_sentences = [' '.join(text) for text in data_new]

vectorizer = CountVectorizer(analyzer='word', max_features=5000);
x_counts = vectorizer.fit_transform(data_new_sentences);
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
model = NMF(n_components=num_topics, init='nndsvd');

model.fit(xtfidf_norm)

def get_nmf_topics(model, n_top_words):
    feat_names = vectorizer.get_feature_names()
    word_dict = {};
    for i in range(num_topics):  
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    return pd.DataFrame(word_dict);

get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,trump,news,hill,debate,impeachment,brexit,republicans,biden,whistleblower,election
1,president,abc,report,candidate,inquiry,johnson,impeachment,joe,complaint,politic
2,donald,broadcast,edition,question,democrats,boris,everything,warren,ukraine,party
3,ukraine,video,morning,stage,house,minister,trump,elizabeth,call,netanyahu
4,call,coverage,controversy,tonight,pelosi,uk,hypocrisy,ukraine,congress,candidate
5,syria,world,welcome,tuesday,nancy,deal,gop,campaign,house,israel
6,china,interview,trump,night,speaker,britain,north,poll,intelligence,voter
7,decision,today,amazon,october,white,prime,carolina,vice,attorney,tunisia
8,washington,fox,re,houston,dem,european,matter,sanders,giuliani,year
9,gop,history,happy,transcript,probe,parliament,defense,age,detail,campaign


In [60]:
#Topic 1: ???
#Topic 2: General news
#Topic 3: ???
#Topic 4: Democratic debates
#Topic 5: Impeachment
#Topic 6: Brexit/EU
#Topic 7: ???
#Topic 8: Progressives
#Topic 9: Giuliani
#Topic 10: Israel?